In [1]:
import meva
from meva.lib.spin import get_pretrained_hmr
import mmcv 
import torch
from meva.utils import image_utils
from utils import imshow
from skimage.transform import resize
import numpy as np

/home/tormod/Documents/APEC/meva/utils/transformations.py:1927: UserWarning: No module named 'meva.utils._transformations'
  warnings.warn(str(err))


In [7]:
vid_folder = 'videos'
vid_name = 'IMG_2139.MOV'
stripped_name = vid_name.split('.')[0]
vid_file = f'{vid_folder}/{vid_name}'
vid = mmcv.video.VideoReader(vid_file)

In [3]:
hmr = get_pretrained_hmr()

====> Loading pre trained: data/meva_data/spin_model_checkpoint.pth.tar


In [4]:
def spin_predict_feature(hmr, img):
    hmr.eval()
    img = image_utils.convert_cvimg_to_tensor(img)
    img = img.unsqueeze(0).float().to('cuda')
    feat = hmr.feature_extractor(img).to('cpu')
    return feat

In [5]:
out_folder = 'features'
for img in vid:
    feat = spin_predict_feature(hmr, img)
    np.save(f'{out_folder}/{stripped_name}/{i:06d}')



RuntimeError: CUDA out of memory. Tried to allocate 30.00 MiB (GPU 0; 1.95 GiB total capacity; 958.89 MiB already allocated; 33.75 MiB free; 1.01 GiB reserved in total by PyTorch)